Extract the data from the output and store in a FITS catalog

In [1]:
from glob import glob
import os, sys
import astropy
from astropy.table import Table, Column
import fitsio
import numpy as np

In [2]:
path = '/global/homes/m/mkwiecie/desi/sv3-clustering'
output_path = '/pscratch/sd/m/mkwiecie/legacydata/output/3962/'
base_cat = 'BGS_BRIGHT_S_clustering.dat.fits'

In [3]:
finished_galaxy_paths = glob(output_path+'*', recursive=False)
print(len(finished_galaxy_paths))

1794


In [4]:
def read_ccds_tractor_sample(galaxy_dir, galaxy, prefix="custom"):
    nccds, tractor, sample = None, None, None
    refcolumn='TARGETID'

    files = os.listdir(galaxy_dir)
    for f in files:
        if 'isfail' in f:
            return None, None


    # samplefile can exist without tractorfile when using --just-coadds
    samplefile = os.path.join(galaxy_dir, "{}-sample.fits".format(galaxy))
    if os.path.isfile(samplefile):
        sample = astropy.table.Table(fitsio.read(samplefile, upper=True))

    tractorfile = os.path.join(
        galaxy_dir, "{}-{}-tractor.fits".format(galaxy, prefix)
    )
    if os.path.isfile(tractorfile):
        tractor = astropy.table.Table(
            fitsio.read(tractorfile, lower=True)
        )  
        
        wt, ws = [], []
        for ii, sid in enumerate(sample[refcolumn]):
            ww = np.where(tractor["ref_id"] == sid)[0]
            if len(ww) > 0:
                wt.append(ww)
                ws.append(ii)
        if len(wt) == 0:
            tractor = None
        else:
            wt = np.hstack(wt)
            ws = np.hstack(ws)
            tractor = tractor[wt]
            sample = sample[ws]
            srt = np.argsort(tractor["flux_r"])[::-1]
            tractor = tractor[srt]
            sample = sample[srt]
            assert np.all(tractor["ref_id"] == sample[refcolumn])
    return tractor, sample

In [5]:
def read_ellipsefit(galaxy, galaxydir, filesuffix='', galaxy_id='', verbose=True,
                    asTable=True):
    """Read the output of write_ellipsefit. Convert the astropy Table into a
    dictionary so we can use a bunch of legacy code.

    """
    if galaxy_id.strip() == '':
        galid = ''
    else:
        galid = '-{}'.format(galaxy_id)
    if filesuffix.strip() == '':
        fsuff = ''
    else:
        fsuff = '-{}'.format(filesuffix)

    ellipsefitfile = os.path.join(galaxydir, '{}{}-ellipse{}.fits'.format(galaxy, fsuff, galid))
        
    if os.path.isfile(ellipsefitfile):
        print(ellipsefitfile,' exists')
        data = Table(fitsio.read(ellipsefitfile))

        # Optionally convert (back!) into a dictionary.
        if asTable:
            return data
        ellipsefit = {}
        for key in data.colnames:
            val = data[key][0]
            ellipsefit[key.lower()] = val # lowercase!
    else:
        print(ellipsefitfile,' doesnt exist')
        return None

    return ellipsefit

In [6]:
tractor_rows = []
sample_rows = []
ellipse_rows = []

for path in finished_galaxy_paths[0:5]:
    galaxydir = path
    galaxy = path.split("/")[-1]
    
    tractor, sample = read_ccds_tractor_sample(galaxydir, galaxy, prefix="custom")
    if tractor is not None:
        tractor_rows.append(tractor)
        sample_rows.append(sample)
        
    ellipse = read_ellipsefit(galaxy,galaxydir,filesuffix="custom",galaxy_id=galaxy,verbose=True,asTable=True)
    if ellipse is not None:
        ellipse_rows.append(ellipse)
    
total_tractor = astropy.table.vstack(tractor_rows)
total_sample = astropy.table.vstack(sample_rows)

/pscratch/sd/m/mkwiecie/legacydata/output/3962/39627877821449852/39627877821449852-custom-ellipse-39627877821449852.fits  exists
/pscratch/sd/m/mkwiecie/legacydata/output/3962/39627793805349506/39627793805349506-custom-ellipse-39627793805349506.fits  exists
/pscratch/sd/m/mkwiecie/legacydata/output/3962/39627805918500370/39627805918500370-custom-ellipse-39627805918500370.fits  exists
/pscratch/sd/m/mkwiecie/legacydata/output/3962/39627758199902938/39627758199902938-custom-ellipse-39627758199902938.fits  exists
/pscratch/sd/m/mkwiecie/legacydata/output/3962/39627878966494779/39627878966494779-custom-ellipse-39627878966494779.fits  exists


In [7]:
col_lkp={}

for col in ellipse_rows[0].columns:
    
    max_len = 1
    max_shape = (1,)
    
    for row in ellipse_rows:
        
        if len(row.columns[col].shape) > 1:    
            
            row_len = row.columns[col].shape[1]
            
            # Store the max length/shape row (for use later when building the table)
            if row_len > max_len:
                
                max_len = row_len
                
                max_shape = row.columns[col].shape

    col_lkp[col] = (row.columns[col].dtype, max_len, max_shape)

In [8]:
from collections import defaultdict
cols = defaultdict(list)
for row in ellipse_rows:
    for col in row.columns:
        cols[col].append(row[col].value[0])


In [10]:
new_cols = dict()

for k, v in cols.items():
    if hasattr(v[0], '__len__') and len(v[0]) > 1:
        maxlen = 1
        for item in v:
            if hasattr(item, '__len__') and len(item) > maxlen:
                is_str = type(item[0]) == np.str_ or type(item[0]) == str
                maxlen = len(item)
        
        if is_str:
            new_cols[k] = v
        else:
            new_items = []
            for item in v:
                new_items.append(np.pad(item.astype(float), (0, maxlen-len(item)), 'constant', constant_values=-1))

            new_cols[k] = new_items
    else:
        new_cols[k] = v
        

In [11]:
total_ellipse = Table()
for k,v in col_lkp.items():
    dat = new_cols[k]
    total_ellipse.add_column(Column(data=dat, name=k, dtype=v[0], shape=v[2], length=len(ellipse_rows)))


In [12]:
print(total_ellipse['ID_CENT'])
print(total_tractor['ref_id'])
print(total_sample['TARGETID'])

     ID_CENT     
-----------------
39627877821449852
39627793805349506
39627805918500370
39627758199902938
39627878966494779
      ref_id     
-----------------
39627877821449852
39627793805349506
39627805918500370
39627758199902938
39627878966494779
     TARGETID    
-----------------
39627877821449852
39627793805349506
39627805918500370
39627758199902938
39627878966494779


In [13]:
total_sample.write('total_sample.fits', format='fits', overwrite=True)
total_tractor.write('total_tractor.fits', format='fits', overwrite=True)
total_ellipse.write('total_ellipse.fits', format='fits', overwrite=True)